In [28]:
import numpy as np
import pandas as pd
import random
import string
import mmh3
import math

### Задание 1

Реализация Bloom-Filter на одной хеш-функции.

In [5]:
class BloomFilter:
    def __init__(self, capacity):
        self.capacity = capacity
        self.array = np.zeros(self.capacity, dtype=int)

    def _compute_hash(self, element):
        return mmh3.hash(element) % self.capacity

    def put(self, element):
        hash_index = self._compute_hash(element)
        self.array[hash_index] += 1

    def get(self, element):
        hash_index = self._compute_hash(element)
        return self.array[hash_index] > 0

    def size(self):
        return np.count_nonzero(self.array)


def generate_random_string(length):
    characters = string.ascii_letters + string.digits
    random_string = ''.join(random.choice(characters) for _ in range(length))
    return random_string

In [6]:
def run_experiment(sizes_bf, sizes_set):
    bloom_filter = BloomFilter(sizes_bf)
    unique_strings = [generate_random_string(10) for _ in range(sizes_set)]
    false_positive_count = 0

    for s in unique_strings:
        if bloom_filter.get(s):
            false_positive_count += 1
        bloom_filter.put(s)

    ones_count = bloom_filter.size()
    return {
        'bf_size': sizes_bf,
        'set_size': sizes_set,
        'fp_count': false_positive_count,
        'ones_count': ones_count
    }

sizes_bf = [8, 64, 1024, 65536, 16777216]
sizes_set = [5, 50, 500, 5000, 5000000]

pd.DataFrame([run_experiment(bf_size, set_size) for bf_size in sizes_bf for set_size in sizes_set])

,bf_size,set_size,fp_count,ones_count
0,8,5,0,5
1,8,50,42,8
2,8,500,492,8
3,8,5000,4992,8
4,8,5000000,4999992,8
5,64,5,0,5
6,64,50,13,37
7,64,500,436,64
8,64,5000,4936,64
9,64,5000000,4999936,64


### Задание 2

Реализация Bloom Filter на k хеш-функций.

In [8]:
class BloomFilter:
    def __init__(self, capacity, k):
        self.capacity = capacity
        self.k = k
        self.array = np.zeros(self.capacity, dtype=int)

    def _compute_hashes(self, element):
        return [(mmh3.hash(element + str(i)) % self.capacity) for i in range(self.k)]

    def put(self, element):
        for hash_index in self._compute_hashes(element):
            self.array[hash_index] += 1

    def get(self, element):
        return all(self.array[hash_index] > 0 for hash_index in self._compute_hashes(element))

    def size(self):
        return np.count_nonzero(self.array)

In [9]:
def run_experiment(sizes_bf, sizes_set, k):
    bloom_filter = BloomFilter(sizes_bf, k)
    unique_strings = [generate_random_string(10) for _ in range(sizes_set)]
    false_positive_count = 0

    for s in unique_strings:
        if bloom_filter.get(s):
            false_positive_count += 1
        bloom_filter.put(s)

    ones_count = bloom_filter.size()
    return {
        'bf_size': sizes_bf,
        'set_size': sizes_set,
        'k': k,
        'fp_count': false_positive_count,
        'ones_count': ones_count
    }


sizes_bf = [8, 64, 1024, 65536, 16777216]
sizes_set = [5, 50, 500, 5000, 5000000]
k_values = [1, 2, 3, 4]

pd.DataFrame([
    run_experiment(bf_size, set_size, k) 
    for bf_size in sizes_bf 
    for set_size in sizes_set 
    for k in k_values
])

,bf_size,set_size,k,fp_count,ones_count
0,8,5,1,2,3
1,8,5,2,1,7
2,8,5,3,2,6
3,8,5,4,2,7
4,8,50,1,42,8
...,...,...,...,...,...
95,16777216,5000,4,0,19984
96,16777216,5000000,1,675269,4324731
97,16777216,5000000,2,387300,7534272
98,16777216,5000000,3,334020,9914639


### Задание 3

Реализация Counting Bloom Filter на k хеш-функций.

In [13]:
class CountingBloomFilter:
    def __init__(self, k, n, cap=1):
        self.k = k
        self.n = n
        self.cap = cap
        self.size = n
        self.counters = np.zeros(self.size, dtype=np.uint8)
    
    def _hashes(self, s):
        return [mmh3.hash(s, i) % self.n for i in range(self.k)]

    def put(self, s):
        for hash_value in self._hashes(s):
            self.counters[hash_value] = min(self.counters[hash_value] + 1, (1 << self.cap) - 1)

    def get(self, s):
        return all(self.counters[hash_value] > 0 for hash_value in self._hashes(s))

    def delete(self, s):
        for hash_value in self._hashes(s):
            if self.counters[hash_value] > 0:
                self.counters[hash_value] -= 1

    def average_count(self):
        return np.sum(self.counters) / self.k

* Для небольших фильтров и больших множеств может возникать переполнение счётчиков, что приведёт к увеличению ложных срабатываний.
* При большем размере фильтра ложные срабатывания должны снизиться, поскольку счётчики будут использоваться реже.
* cap = 3 означает, что каждый счётчик может хранить значения от 0 до 7.

In [18]:
def run_experiment(n, set_size, k, cap=3):
    cbf = CountingBloomFilter(k=k, n=n, cap=cap)
    unique_strings = [generate_random_string(10) for _ in range(set_size)]
    false_positive_count = 0

    for s in unique_strings:
        cbf.put(s)

    for s in unique_strings:
        if cbf.get(s):
            false_positive_count += 1
            
    ones_count = cbf.average_count()

    return {
        'n': n,
        'set_size': set_size,
        'k': k,
        'fp_count': false_positive_count,
        'ones_count': ones_count
    }

results = []
for n in [8, 64, 1024, 64 * 1024]:
    for set_size in [10, 100, 1000, 100000]:
        for k in [1, 2, 3, 4]:
            result = run_experiment(n, set_size, k)
            results.append(result)

pd.DataFrame(results)

,n,set_size,k,fp_count,ones_count
0,8,10,1,10,10.000000
1,8,10,2,10,10.000000
2,8,10,3,10,10.000000
3,8,10,4,10,9.250000
4,8,100,1,100,56.000000
...,...,...,...,...,...
59,65536,1000,4,1000,1000.000000
60,65536,100000,1,100000,99985.000000
61,65536,100000,2,100000,99327.000000
62,65536,100000,3,100000,96287.333333


### Задание 4

Реализация HyperLogLog.

In [26]:
class HyperLogLog:
    def __init__(self, k, n):
        self.k = k
        self.n = n
        self.buckets = np.zeros(self.n, dtype=np.uint8)

    def _hash(self, s):
        hash_value = mmh3.hash(s, 42)
        return format(hash_value, '032b')

    def put(self, s):
        hash_bin = self._hash(s)
        bucket_index = int(hash_bin[:self.k], 2)
        remaining_bits = hash_bin[self.k:]
        leading_zeros = len(remaining_bits) - len(remaining_bits.lstrip('0'))
        self.buckets[bucket_index] = max(self.buckets[bucket_index], leading_zeros + 1)

    def est_size(self):
        Z = 0
        for bucket in self.buckets:
            Z += 2 ** -bucket 
        
        alpha_m = 0.7213 / (1 + 1.079 / self.n)
        raw_estimate = alpha_m * self.n ** 2 / Z
        
        if raw_estimate <= 2.5 * self.n:
            V = sum(1 for x in self.buckets if x == 0)
            if V > 0:
                raw_estimate = self.n * math.log(self.n / V)
        
        return int(raw_estimate)

* Проверим, как алгоритм ведет себя при малом количестве элементов.
* Проверим масштабируемость алгоритма и его точность при увеличении числа элементов.
* Добавим набор уникальных элементов, а затем повторим их несколько раз, чтобы убедиться, что HyperLogLog не учитывает повторяющиеся элементы и оценка количества уникальных элементов остаётся постоянной.
* Проведем эксперимент с различными значениями k, чтобы понять, как изменение количества бакетов влияет на точность алгоритма.
* Используем короткие строки, которые могут легко хешироваться в одно и то же значение, чтобы проверить устойчивость алгоритма к хеш-коллизиям.

In [29]:
def run_hll_experiment(k, n, num_elements, add_repeats=False):
    hll = HyperLogLog(k=k, n=n)
    unique_strings = {generate_random_string(10) for _ in range(num_elements)}

    for s in unique_strings:
        hll.put(s)

    if add_repeats:
        for s in unique_strings:
            hll.put(s)

    estimated_size = hll.est_size()

    return {
        'k': k,
        'n': n,
        'actual_size': len(unique_strings),
        'estimated_size': estimated_size,
        'error': abs(len(unique_strings) - estimated_size)
    }

k_values = [8, 10, 12]
n_values = [2**k for k in k_values]
element_counts = [5, 50, 500, 10000]

results = []

for k, n in zip(k_values, n_values):
    for num_elements in element_counts:
        result = run_hll_experiment(k, n, num_elements)
        results.append(result)

for k, n in zip(k_values, n_values):
    for num_elements in element_counts:
        result = run_hll_experiment(k, n, num_elements, add_repeats=True)
        results.append(result)

pd.DataFrame(results)

C:\Users\asmir\AppData\Local\Temp\ipykernel_8384\2998940640.py:21: RuntimeWarning: overflow encountered in scalar negative
  Z += 2 ** -bucket


,k,n,actual_size,estimated_size,error
0,8,256,5,5,0
1,8,256,50,51,1
2,8,256,500,1176,676
3,8,256,10000,47072,37072
4,10,1024,5,5,0
5,10,1024,50,50,0
6,10,1024,500,474,26
7,10,1024,10000,755541,745541
8,12,4096,5,5,0
9,12,4096,50,49,1


* Точность HyperLogLog очень высока для небольших множеств (до 50 элементов) даже при малом количестве бакетов.
* Ошибка начинает расти при увеличении фактического количества элементов, особенно при малом количестве бакетов (n = 256), что вызывает завышенные оценки.
* Увеличение количества бакетов снижает ошибку, особенно для более крупных наборов данных.
* Количество хеш-функций (k) влияет на точность при работе с большими множествами. Меньшие значения k (например, k=8) при высоких значениях n дают более точные результаты.